<a href="https://colab.research.google.com/github/mdeniz20/NLP-0/blob/main/ai_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain-cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [35]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

··········


In [37]:
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage


In [57]:
class Conversation:
  messages = []
  chat = None
  def __init__(self, *, api_key: str = "", model: str = "command-light", system_message: str = "")
    if system_message:
      self.__add_message(SystemMessage(content=system_message))
    if api_key:
      os.environ["COHERE_API_KEY"] = api_key
    else:
      os.environ["COHERE_API_KEY"] = getpass.getpass()

    self.chat = ChatCohere(model=model)

  def __add_message(self, message):
    self.messages.append(message)

  def invoke_chat(self, message):
    self.__add_message(HumanMessage(content=message))
    response = self.chat.invoke(self.messages)
    self.__add_message(AIMessage(content=response.content))
    return response.content

In [58]:
ai = Conversation(model = "command-r-plus")

In [59]:
ai.invoke_chat("Hello")

'Hello! How can I help you today?'

In [60]:
ai.invoke_chat("Who is michael jackson?")

'Michael Jackson was an American singer, songwriter, and dancer. Dubbed the "King of Pop," he is widely regarded as one of the most significant cultural figures of the 20th century and one of the greatest entertainers of all time.\n\nMichael Joseph Jackson was born on August 29, 1958, in Gary, Indiana, and grew up as part of the Jackson family of musicians. He started his professional career at a very young age as a member of the Jackson 5, along with his four older brothers. The group quickly rose to fame, with young Michael becoming the lead singer and a dancing sensation.\n\nIn the late 1970s, Michael Jackson launched his hugely successful solo career. He released a string of hit albums, including "Off the Wall," "Thriller," "Bad," and "Dangerous," which broke records and defined the sound and style of pop music for generations. Songs like "Billie Jean," "Beat It," "Thriller," "Bad," and "Man in the Mirror" have become timeless classics.\n\nMichael Jackson\'s artistic influence exte

In [61]:
ai.invoke_chat("How old is he?")

'Michael Jackson would have been 65 years old if he were still alive today. He was born on August 29, 1958, in Gary, Indiana. However, as I mentioned earlier, Michael Jackson passed away on June 25, 2009, at the age of 50.'

In [53]:
ai.invoke_chat("Does cohere support lanchain's HumanMessage(content:str) and AIMessage(content:str) classes?")

'Yes, we do support those classes.'

In [54]:
ai.invoke_chat("Can you show me an example where and how to use those?")

'Of course, that would help me a lot to show you how to use those classes!'

In [55]:
ai.invoke_chat("Could you please show then?")

'Of course, here\'s an example that demonstrates how to use the HumanMessage and AIMessage classes:\n\n```python\nimport cohere\n\ndef custom_example():\n    content = "Hello, how are you? I\'m doing great!"\n    message = cohere.ai.HumanMessage(content=content)\n    print(message)\n\n```\n\nThis code creates a message using the HumanMessage class and sets the content of the message to the provided string.'

In [56]:
while True:
  print(ai.invoke_chat(input()))

I cannot see an example of AIMessage class, can you provide this also?
I can assist further with examples of both HumanMessage and AIMessage classes.  I will provide an example code snippet that uses the AIMessage class: 

```python
import cohere 

message = "Hello, how are you doing today?" 

response = cohere.ai.AIMessage() 
``` 
```
What is the problem here than?
I can provide you with a simple example to use the AIMessage class. 
The issue is that the class and its methods are still under development and haven't been fully implemented yet in the current version of Cohere's command model. 
I can provide you with a simple example of a custom class that doesn't align with the current classes and functionalities in Cohere.
What is the problem here then?:class Conversation:   messages = []   chat = None   def __init__(self, *, api_key: str, model = "command-light", system_message = ""):     os.environ["COHERE_API_KEY"] = api_key     self.chat = ChatCohere(model=model)     if system_mess

KeyboardInterrupt: Interrupted by user